# Imports

In [183]:
import numpy as np
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from pandas.plotting import scatter_matrix
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import confusion_matrix, f1_score, average_precision_score, classification_report, fbeta_score, accuracy_score
from sklearn.feature_selection import RFECV

from statsmodels.stats.outliers_influence import variance_inflation_factor

import optuna
import statsmodels.api as sm
from boruta import BorutaPy

import custom_map

In [184]:
import importlib

importlib.reload(custom_map)

<module 'custom_map' from '/Users/dominikmika/PycharmProjects/Ridge-hillclimbing/custom_map.py'>

# Setup

In [185]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
bmi_median = data['bmi'].median()
data['bmi'] = data['bmi'].fillna(bmi_median)
data.info()

categorical_features = data.select_dtypes(['object']).columns.tolist()
numerical_features = data.select_dtypes(['float64', 'int64']).columns.drop('id')

data = pd.get_dummies(data, columns=categorical_features, drop_first=True, dtype=float)
data = data.drop('id', axis=1)

target = "stroke"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [186]:
X = data.drop(columns=[target])
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, shuffle=True)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [187]:
## hard coding because of stochasticity

all_features = ["age", "hypertension", "heart_disease", "avg_glucose_level",
                "bmi", "gender_Male", "gender_Other", "ever_married_Yes",
                "work_type_Never_worked", "work_type_Private", "work_type_Self-employed",
                "work_type_children", "Residence_type_Urban",
                "smoking_status_formerly smoked", "smoking_status_never smoked",
                "smoking_status_smokes"]

boruta_features = ["age", "avg_glucose_level", "bmi"]

corr_features = ["age", "heart_disease", "avg_glucose_level", "hypertension",
                 "ever_married_Yes", "smoking_status_formerly smoked",
                 "work_type_Self-employed"]

mi_features = ["age", "hypertension", "gender_Other",
               "work_type_Private", "smoking_status_formerly smoked"]

rfe_features = ["age", "hypertension", "heart_disease", "avg_glucose_level",
                "bmi", "work_type_Never_worked", "work_type_children",
                "Residence_type_Urban", "smoking_status_never smoked",
                "smoking_status_smokes"]

FEATURE_SETS = {
    "all": all_features,
    "boruta": boruta_features,
    "correlation": corr_features,
    "mi": mi_features,
    "rfe": rfe_features
}

In [188]:
storage_url = "sqlite:///optuna_studies.db"
cv = StratifiedKFold(5, shuffle=True, random_state=42)

MODELS = [
    "logreg", "knn", "svm", "gnb", "dt",
    "rf", "ada", "gb", "extra",
    "lgbm", "xgb", "cat"
]

SMOTE_MODELS = {"logreg", "knn", "svm", "gnb"}

# Results

In [189]:
def find_best_f1_threshold(model, X, y, thresholds=np.linspace(0.01, 0.5, 50)):
    proba = model.predict_proba(X)[:, 1]

    best_f1 = 0.0
    best_thr = 0.5

    for t in thresholds:
        y_pred = (proba >= t).astype(int)
        f1 = f1_score(y, y_pred)

        if f1 > best_f1:
            best_f1 = f1
            best_thr = t

    return best_thr, best_f1

In [190]:
for feature_name, feature_list in FEATURE_SETS.items():

    X_train_sel = X_train[feature_list]

    for model_name in MODELS:

        study_name = f"{model_name}_{feature_name}_prob"

        with open(f"models/prob_f1/{study_name}.pkl", "rb") as f:
            artifact = pickle.load(f)

        model = artifact["model"]

        best_thr, best_f1 = find_best_f1_threshold(
            model, X_train_sel, y_train
        )

        artifact = {
            "model": model,
            "threshold": best_thr,
            "f1_train": best_f1
        }

        with open(f"models/{study_name}.pkl", "wb") as f:
            pickle.dump(artifact, f)

        print(
            f"SAVED {study_name} | "
            f"thr={best_thr:.2f} | "
            f"F1_train={best_f1:.4f}"
        )

SAVED logreg_all_prob | thr=0.50 | F1_train=0.2193
SAVED knn_all_prob | thr=0.50 | F1_train=0.8556
SAVED svm_all_prob | thr=0.14 | F1_train=0.4575
SAVED gnb_all_prob | thr=0.37 | F1_train=0.1063
SAVED dt_all_prob | thr=0.44 | F1_train=0.2522
SAVED rf_all_prob | thr=0.50 | F1_train=0.3361
SAVED ada_all_prob | thr=0.42 | F1_train=0.2778
SAVED gb_all_prob | thr=0.31 | F1_train=0.8511
SAVED extra_all_prob | thr=0.50 | F1_train=0.2481
SAVED lgbm_all_prob | thr=0.50 | F1_train=0.4726
SAVED xgb_all_prob | thr=0.27 | F1_train=0.7090
SAVED cat_all_prob | thr=0.50 | F1_train=0.5819
SAVED logreg_boruta_prob | thr=0.50 | F1_train=0.2184
SAVED knn_boruta_prob | thr=0.01 | F1_train=0.4083
SAVED svm_boruta_prob | thr=0.13 | F1_train=0.2723
SAVED gnb_boruta_prob | thr=0.11 | F1_train=0.2407
SAVED dt_boruta_prob | thr=0.46 | F1_train=0.2254
SAVED rf_boruta_prob | thr=0.50 | F1_train=0.3209
SAVED ada_boruta_prob | thr=0.32 | F1_train=0.2825
SAVED gb_boruta_prob | thr=0.34 | F1_train=0.8223
SAVED extra_b

In [191]:
results = pd.DataFrame(
    index=MODELS,
    columns=FEATURE_SETS.keys(),
    dtype=float
)

for feature_name, feature_list in FEATURE_SETS.items():

    X_test_sel = X_test[feature_list]

    for model_name in MODELS:

        study_name = f"{model_name}_{feature_name}_prob"

        with open(f"models/{study_name}.pkl", "rb") as f:
            artifact = pickle.load(f)

        model = artifact["model"]
        best_thr = artifact["threshold"]

        proba_test = model.predict_proba(X_test_sel)[:, 1]
        y_pred_test = (proba_test >= best_thr).astype(int)

        f1 = f1_score(y_test, y_pred_test)
        results.loc[model_name, feature_name] = f1

        print(
            f"{model_name}_{feature_name} | "
            f"thr={best_thr:.2f} | "
            f"F1_test={f1:.4f}"
        )

logreg_all | thr=0.50 | F1_test=0.2639
knn_all | thr=0.50 | F1_test=0.8632
svm_all | thr=0.14 | F1_test=0.5385
gnb_all | thr=0.37 | F1_test=0.1063
dt_all | thr=0.44 | F1_test=0.2486
rf_all | thr=0.50 | F1_test=0.4201
ada_all | thr=0.42 | F1_test=0.3176
gb_all | thr=0.31 | F1_test=0.9032
extra_all | thr=0.50 | F1_test=0.2736
lgbm_all | thr=0.50 | F1_test=0.4865
xgb_all | thr=0.27 | F1_test=0.7500
cat_all | thr=0.50 | F1_test=0.6143
logreg_boruta | thr=0.50 | F1_test=0.2493
knn_boruta | thr=0.01 | F1_test=0.3982
svm_boruta | thr=0.13 | F1_test=0.3271
gnb_boruta | thr=0.11 | F1_test=0.2846
dt_boruta | thr=0.46 | F1_test=0.2432
rf_boruta | thr=0.50 | F1_test=0.3482
ada_boruta | thr=0.32 | F1_test=0.3448
gb_boruta | thr=0.34 | F1_test=0.8632
extra_boruta | thr=0.50 | F1_test=0.2866
lgbm_boruta | thr=0.50 | F1_test=0.3154
xgb_boruta | thr=0.34 | F1_test=0.5872
cat_boruta | thr=0.50 | F1_test=0.4868
logreg_correlation | thr=0.50 | F1_test=0.2217
knn_correlation | thr=0.01 | F1_test=0.4186
svm

In [192]:
results = results.sort_values(
    by=results.columns.tolist(),
    ascending=False
)

results

,all,boruta,correlation,mi,rfe
gb,0.903226,0.863158,0.851064,0.356322,0.822222
knn,0.863158,0.398190,0.418605,0.280193,0.913043
xgb,0.750000,0.587156,0.872340,0.371585,0.893617
cat,0.614286,0.486772,0.475138,0.313333,0.539877
svm,0.538462,0.327103,0.427673,0.315152,0.424242
lgbm,0.486486,0.315436,0.389831,0.282353,0.494382
rf,0.420091,0.348178,0.436782,0.312500,0.314685
ada,0.317647,0.344828,0.309392,0.346821,0.327273
extra,0.273616,0.286645,0.314607,0.278638,0.264264
logreg,0.263930,0.249275,0.221687,0.259366,0.259587


In [193]:
def load_all_model_predictions(
    MODELS, FEATURE_SETS,
    X_train, y_train,
    X_test
):
    train_preds = []
    test_preds = []
    model_names = []

    for feature_name, feature_list in FEATURE_SETS.items():
        X_train_sel = X_train[feature_list]
        X_test_sel = X_test[feature_list]

        for model_name in MODELS:
            study_name = f"{model_name}_{feature_name}_prob"

            with open(f"models/{study_name}.pkl", "rb") as f:
                artifact = pickle.load(f)

            model = artifact["model"]

            p_train = model.predict_proba(X_train_sel)[:, 1]
            p_test = model.predict_proba(X_test_sel)[:, 1]

            train_preds.append(p_train)
            test_preds.append(p_test)
            model_names.append(study_name)

    P_train = np.column_stack(train_preds)
    P_test = np.column_stack(test_preds)

    return P_train, P_test, model_names


In [194]:
def extract_feature_set(model_name, FEATURE_SETS):
    for fs in FEATURE_SETS:
        if model_name.endswith(f"_{fs}"):
            return fs
    raise ValueError(f"Nie można dopasować feature setu do modelu: {model_name}")

def evaluate_single_model(
    artifact,
    model_name,
    X_train, y_train,
    X_test, y_test,
    FEATURE_SETS
):
    model = artifact["model"]
    threshold = artifact.get("threshold", 0.5)

    feature_set = extract_feature_set(model_name, FEATURE_SETS)
    features = FEATURE_SETS[feature_set]

    Xtr = X_train[features]
    Xte = X_test[features]

    train_proba = model.predict_proba(Xtr)[:, 1]
    test_proba = model.predict_proba(Xte)[:, 1]

    train_pred = (train_proba >= threshold).astype(int)
    test_pred = (test_proba >= threshold).astype(int)

    return (
        {
            "accuracy": accuracy_score(y_train, train_pred),
            "f1": f1_score(y_train, train_pred),
            "avg_precision": average_precision_score(y_train, train_proba),
        },
        {
            "accuracy": accuracy_score(y_test, test_pred),
            "f1": f1_score(y_test, test_pred),
            "avg_precision": average_precision_score(y_test, test_proba),
        },
        threshold
    )

def evaluate_saved_models(
    model_paths,
    X_train, y_train,
    X_test, y_test,
    MODELS,
    FEATURE_SETS
):
    rows = []

    for name, path in model_paths.items():

        with open(path, "rb") as f:
            artifact = pickle.load(f)

        # =========================
        # ENSEMBLE HILL CLIMBING
        # =========================
        if isinstance(artifact, dict) and artifact.get("type") == "hill_climbing_ensemble":

            weights = artifact["weights"]
            threshold = artifact["threshold"]

            P_train, P_test, model_names = load_all_model_predictions(
                MODELS, FEATURE_SETS,
                X_train, y_train,
                X_test
            )

            train_proba = P_train @ weights
            test_proba = P_test @ weights

            train_pred = (train_proba >= threshold).astype(int)
            test_pred = (test_proba >= threshold).astype(int)

            train_scores = {
                "accuracy": accuracy_score(y_train, train_pred),
                "f1": f1_score(y_train, train_pred),
                "avg_precision": average_precision_score(y_train, train_proba),
            }

            test_scores = {
                "accuracy": accuracy_score(y_test, test_pred),
                "f1": f1_score(y_test, test_pred),
                "avg_precision": average_precision_score(y_test, test_proba),
            }

        # =========================
        # POJEDYNCZY MODEL
        # =========================
        else:
            train_scores, test_scores, threshold = evaluate_single_model(
                artifact,
                name,
                X_train, y_train,
                X_test, y_test,
                FEATURE_SETS
            )

        rows.append({
            "model": name,

            "train_f1": train_scores["f1"],
            "test_f1": test_scores["f1"],

            "train_accuracy": train_scores["accuracy"],
            "test_accuracy": test_scores["accuracy"],

            "train_avg_precision": train_scores["avg_precision"],
            "test_avg_precision": test_scores["avg_precision"],

            "threshold": threshold
        })

    return (
        pd.DataFrame(rows)
        .set_index("model")
        .sort_values("test_avg_precision", ascending=False)
    )



In [195]:
MODEL_PATHS = {
    "knn_all": "models/prob_f1/knn_all_prob.pkl",
    "knn_rfe": "models/prob_f1/knn_rfe_prob.pkl",

    "gb_all": "models/prob_f1/gb_all_prob.pkl",
    "gb_boruta": "models/prob_f1/gb_boruta_prob.pkl",
    "gb_correlation": "models/prob_f1/gb_correlation_prob.pkl",
    "gb_rfe": "models/prob_f1/gb_rfe_prob.pkl",

    "xgb_all": "models/prob_f1/xgb_all_prob.pkl",
    "xgb_correlation": "models/prob_f1/xgb_correlation_prob.pkl",
    "xgb_rfe": "models/prob_f1/xgb_rfe_prob.pkl",

    "cat_all": "models/prob_f1/cat_all_prob.pkl",
    "ensemble_hill": "models/hc/hc_ensemble.pkl",
    "ensemble_hill_2": "models/hc/hc_ensemble_v2.pkl",
    "ensemble_hill_3": "models/hc/hc_ensemble_v3.pkl"
}

results = evaluate_saved_models(
    MODEL_PATHS,
    X_train, y_train,
    X_test, y_test,
    MODELS=MODELS,
    FEATURE_SETS=FEATURE_SETS
)

results = results.sort_values("test_f1", ascending=False)
results

,train_f1,test_f1,train_accuracy,test_accuracy,train_avg_precision,test_avg_precision,threshold
model,,,,,,,
ensemble_hill_3,0.874317,0.891304,0.988748,0.990215,0.889573,0.899478,0.50
ensemble_hill,0.877384,0.891304,0.988992,0.990215,0.888836,0.896511,0.50
gb_all,0.847368,0.875000,0.985812,0.988258,0.858587,0.903001,0.27
ensemble_hill_2,0.833760,0.872340,0.984100,0.988258,0.876639,0.893386,0.50
gb_boruta,0.815584,0.863158,0.982632,0.987280,0.852495,0.841776,0.31
gb_correlation,0.826873,0.845361,0.983611,0.985323,0.837895,0.851889,0.28
xgb_correlation,0.800995,0.845361,0.980431,0.985323,0.814479,0.807891,0.33
xgb_rfe,0.803828,0.843137,0.979941,0.984344,0.845614,0.890719,0.12
gb_rfe,0.807198,0.817204,0.981654,0.983366,0.843277,0.818457,0.29


Wnioski:
- Wagi bardzo waźny hiperparametr
- Dobry ensembling zawsze polepsza wyniki

Discussion:
- Oversampling poprzez imbalanced learn
- Avarage precision score zamiast f1 dla recall < precision importance dla nie zbalansowanych klas
